# Dependency Installation and Repository Cloning

### Run it if you're using this notebook in Google Colab

In [ ]:
!git clone 'https://github.com/dakopecky/nlp-course-itmo.git'

%cd nlp-course-itmo
%cd hw1

!pip install poetry
!poetry config virtualenvs.create false
!poetry install --no-ansi

# Dataset Loading and Preprocessing

Import deps

In [ ]:
# This code includes software developed by the following open-source projects:
# - NLTK (License: Apache License 2.0, Authors: Steven Bird, Edward Loper, Ewan Klein)
# - preprocessor (License: MIT, Authors: Saurabh Mathur)
# - Corus (License: MIT, Authors: Denis Emelin)
# - Pymorphy2 (License: MIT, Authors: Mikhail Korobov)
# - Jupyter Notebook (License: Modified BSD License, Authors: Project Jupyter)
# For the full license information, please see the `licenses` directory.


import nltk
import preprocessor as p
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from corus import load_mokoron
import pymorphy2

Download the "Mokoron Russian Twitter Corpus" dataset

In [ ]:
!gdown '1H_xuXdB9_YkIZJMwr-UGY1cbMLJt4JgK' -O 'db.sql'

Load the dataset

In [ ]:
path = 'db.sql'
records = load_mokoron(path)

Extract text

In [ ]:
N = 100000
texts = [record.text for _, record in zip(range(N), records)]

Load resources for nltk

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

Lowercase the text

In [ ]:
texts = [text.lower() for text in texts]

Remove @tags, links and \\n noise

Text before the cleanup

In [ ]:
texts[:5]

['пропавшая в хабаровске школьница почти сутки провела в яме у коллектор',
 'лента, я сегодня полгода дирекшионеееер! с:\\nхотя все равно никто не поздравит лол',
 'царствие божие внутрь вас есть.',
 'rt @twitregion: ученые: кофе приносит намного больше вреда, чем пиво',
 'http://t.co/zzzwylsmon справка по адаптации 5 классников']

In [ ]:
p.set_options(p.OPT.URL, p.OPT.MENTION)
texts = [p.clean(text.replace('\\n', ' ')) for text in texts]

Text after

In [ ]:
texts[:5]

['пропавшая в хабаровске школьница почти сутки провела в яме у коллектор',
 'лента, я сегодня полгода дирекшионеееер! с: хотя все равно никто не поздравит лол',
 'царствие божие внутрь вас есть.',
 'rt : ученые: кофе приносит намного больше вреда, чем пиво',
 'справка по адаптации 5 классников']

Tokenize the text

In [ ]:
tknzr = TweetTokenizer()
tokenized_texts = [tknzr.tokenize(text) for text in texts]

In [ ]:
tokenized_texts[:5]

[['пропавшая',
  'в',
  'хабаровске',
  'школьница',
  'почти',
  'сутки',
  'провела',
  'в',
  'яме',
  'у',
  'коллектор'],
 ['лента',
  ',',
  'я',
  'сегодня',
  'полгода',
  'дирекшионееер',
  '!',
  'с',
  ':',
  'хотя',
  'все',
  'равно',
  'никто',
  'не',
  'поздравит',
  'лол'],
 ['царствие', 'божие', 'внутрь', 'вас', 'есть', '.'],
 ['rt',
  ':',
  'ученые',
  ':',
  'кофе',
  'приносит',
  'намного',
  'больше',
  'вреда',
  ',',
  'чем',
  'пиво'],
 ['справка', 'по', 'адаптации', '5', 'классников']]

Remove punctuation

In [ ]:
exclude_punctuations = {',', '.', ';', ':', '"', "'", "{", "}", "[", "]", "*", "-", "—", "_", "/", "\\", "&"}
filtered_texts = [[token for token in text if token not in exclude_punctuations] for text in tokenized_texts]

In [ ]:
filtered_texts[:5]

[['пропавшая',
  'в',
  'хабаровске',
  'школьница',
  'почти',
  'сутки',
  'провела',
  'в',
  'яме',
  'у',
  'коллектор'],
 ['лента',
  'я',
  'сегодня',
  'полгода',
  'дирекшионееер',
  '!',
  'с',
  'хотя',
  'все',
  'равно',
  'никто',
  'не',
  'поздравит',
  'лол'],
 ['царствие', 'божие', 'внутрь', 'вас', 'есть'],
 ['rt',
  'ученые',
  'кофе',
  'приносит',
  'намного',
  'больше',
  'вреда',
  'чем',
  'пиво'],
 ['справка', 'по', 'адаптации', '5', 'классников']]

Remove stop-words

In [ ]:
russian_stopwords = stopwords.words("russian")
filtered_texts = [[word for word in text if word not in russian_stopwords] for text in filtered_texts]

In [ ]:
filtered_texts[:5]

[['пропавшая',
  'хабаровске',
  'школьница',
  'сутки',
  'провела',
  'яме',
  'коллектор'],
 ['лента',
  'сегодня',
  'полгода',
  'дирекшионееер',
  '!',
  'хотя',
  'равно',
  'никто',
  'поздравит',
  'лол'],
 ['царствие', 'божие', 'внутрь'],
 ['rt', 'ученые', 'кофе', 'приносит', 'намного', 'вреда', 'пиво'],
 ['справка', 'адаптации', '5', 'классников']]

Lemmatize the text

In [ ]:
morph = pymorphy2.MorphAnalyzer()
lemmatized_texts = [[morph.parse(word)[0].normal_form for word in text] for text in filtered_texts]

In [ ]:
lemmatized_texts[:5]

[['пропасть',
  'хабаровск',
  'школьница',
  'сутки',
  'провести',
  'яма',
  'коллектор'],
 ['лента',
  'сегодня',
  'полгода',
  'дирекшионееер',
  '!',
  'хотя',
  'равно',
  'никто',
  'поздравить',
  'лола'],
 ['царствие', 'божий', 'внутрь'],
 ['rt', 'учёный', 'кофе', 'приносить', 'намного', 'вред', 'пиво'],
 ['справка', 'адаптация', '5', 'классник']]